In [19]:
import random
import pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nltk.tokenize import RegexpTokenizer

import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Activation
from tensorflow.keras.optimizers import RMSprop

In [20]:
# Load text data from a CSV file
text_df = pd.read_csv("news.csv")
text = list(text_df.text.values)
joined_text = " ".join(text)

# Save the joined text to a text file
with open("joined_text.txt", "w", encoding="utf-8") as f:
    f.write(joined_text)

In [21]:
# Take a partial portion of the text for tokenization
partial_text = joined_text[:10000]

In [22]:
# Tokenize the partial text using a regular expression tokenizer
tokenizer = RegexpTokenizer(r"\w+")
tokens = tokenizer.tokenize(partial_text.lower())

In [23]:
# Identify unique tokens and create an index mapping
unique_tokens = np.unique(tokens)
unique_token_index = {token: index for index, token in enumerate(unique_tokens)}

In [24]:
print(unique_tokens)

['2016' '2020' '5' '60' 'a' 'abc' 'abcpolitics' 'abedin' 'about'
 'aboutface' 'abuses' 'accused' 'accusing' 'act' 'ad' 'admits' 'ads'
 'afraid' 'after' 'afternoon' 'against' 'age' 'agency' 'agents' 'ago'
 'ahead' 'alive' 'all' 'allegations' 'allies' 'allowed' 'already' 'also'
 'amendment' 'americans' 'an' 'and' 'announced' 'anthony' 'any' 'anywhere'
 'apolitical' 'appearance' 'appeared' 'appearing' 'appeaser' 'approach'
 'are' 'around' 'arrogant' 'article' 'as' 'asked' 'assault' 'assaulting'
 'assaults' 'associates' 'assume' 'at' 'attack' 'attacked' 'attacking'
 'away' 'awkward' 'awkwardly' 'back' 'backed' 'bad' 'badly' 'batch'
 'bathroom' 'be' 'becoming' 'beds' 'been' 'before' 'behavior' 'behind'
 'being' 'belief' 'believes' 'believing' 'better' 'between' 'bigger'
 'bigotry' 'bizarre' 'boldly' 'born' 'boston' 'bragged' 'breathing'
 'breeze' 'breezy' 'bribery' 'bring' 'bureau' 'buried' 'but' 'by' 'cable'
 'calling' 'came' 'campaign' 'can' 'candidate' 'cards' 'career' 'careers'
 'carvil

In [25]:
# Define the number of input words and initialize input and output data
n_words = 10
input_words = []
next_word = []

# Create input sequences and corresponding next words
for i in range(len(tokens) - n_words):
    input_words.append(tokens[i:i + n_words])
    next_word.append(tokens[i + n_words])

In [26]:
# Initialize binary matrices for input and output data
X = np.zeros((len(input_words), n_words, len(unique_tokens)), dtype=bool)  # for each sample, n input words and then a boolean for each possible next word
y = np.zeros((len(next_word), len(unique_tokens)), dtype=bool)  # for each sample a boolean for each possible next word

In [27]:
# Populate the binary matrices with one-hot encoded data
for i, words in enumerate(input_words):
    for j, word in enumerate(words):
        X[i, j, unique_token_index[word]] = 1
    y[i, unique_token_index[next_word[i]]] = 1

In [28]:
# Define the neural network model
model = Sequential()
model.add(LSTM(128, input_shape=(n_words, len(unique_tokens)), return_sequences=True))
model.add(LSTM(128))
model.add(Dense(len(unique_tokens)))
model.add(Activation("softmax"))

In [45]:
# Define the optimizer and compile the model
optimizer = RMSprop(learning_rate=0.1)
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

# Train the model and store the training history
history = model.fit(X, y, batch_size=128, epochs=10, shuffle=True).history

Epoch 1/10
14/14 [==============================] - 3s 74ms/step - loss: 43.4119 - accuracy: 0.0235
Epoch 2/10
14/14 [==============================] - 1s 71ms/step - loss: 34.3005 - accuracy: 0.0286
Epoch 3/10
14/14 [==============================] - 1s 73ms/step - loss: 29.2202 - accuracy: 0.0200
Epoch 4/10
14/14 [==============================] - 1s 72ms/step - loss: 25.3772 - accuracy: 0.0223
Epoch 5/10
14/14 [==============================] - 1s 72ms/step - loss: 22.1019 - accuracy: 0.0297
Epoch 6/10
14/14 [==============================] - 1s 72ms/step - loss: 19.4736 - accuracy: 0.0149
Epoch 7/10
14/14 [==============================] - 1s 73ms/step - loss: 17.3074 - accuracy: 0.0320
Epoch 8/10
14/14 [==============================] - 1s 75ms/step - loss: 15.2882 - accuracy: 0.0366
Epoch 9/10
14/14 [==============================] - 1s 77ms/step - loss: 13.2634 - accuracy: 0.0269
Epoch 10/10
14/14 [==============================] - 1s 76ms/step - loss: 11.3580 - accuracy: 0.0189

In [46]:
# Save the trained model and training history
model.save("text_gen_model1.h5")
with open("history1.p", "wb") as f:
    pickle.dump(history, f)

In [47]:
# Loading the model / Start here if the model is already trained
model = load_model("text_gen_model1.h5")
history = pickle.load(open("history1.p", "rb"))

In [48]:
# Function to predict the next word(s) given an input text
def predict_next_word(input_text, n_best):
    input_text = input_text.lower()
    X = np.zeros((1, n_words, len(unique_tokens)))
    for i, word in enumerate(input_text.split()):
        X[0, i, unique_token_index[word]] = 1
        
    predictions = model.predict(X)[0]
    return np.argpartition(predictions, -n_best)[-n_best:]

In [49]:
# Example usage of the predict_next_word function
possible = predict_next_word("She will have to look into this thing and she", 5)

1/1 [==============================] - 0s 475ms/step


In [50]:
for idx in possible:
    print(unique_tokens[idx])

but
2016
has
of
fbi


In [51]:
# Function to generate text given an initial input and desired number of words
def generate_text(input_text, n_words, creativity=3):
    word_sequence = input_text.split()
    current = 0
    for _ in range(n_words):
        sub_sequence = " ".join(tokenizer.tokenize(" ".join(word_sequence).lower())[current:current+n_words])
        try:
            choice = unique_tokens[random.choice(predict_next_word(sub_sequence, creativity))]
        except:
            choice = random.choice(unique_tokens)
        word_sequence.append(choice)
        current += 1
    return " ".join(word_sequence)

In [52]:
# Generate text using the generate_text function
generate_text("He must have one thing that I am into the", 100, 10)

1/1 [==============================] - 0s 22ms/step


1/1 [==============================] - 0s 16ms/step


'He must have one thing that I am into the while mob underway carville poses greenfield assume desperately but new unprecedented head but 2016 fbi 2016 unprecedented head 2016 new 2016 new head unprecedented fbi an fbi fbi an email of fbi has new 2016 has new new an email has has email 2016 new of 2016 unprecedented 2016 new head has email 2016 an new new has new unprecedented of fbi unprecedented of email 2016 email an an has of fbi email email unprecedented new of 2016 email unprecedented head but email head fbi 2016 2016 of head head unprecedented has fbi but email an 2016 2016 new unprecedented'